In [3]:
# Wine quality prediction using random forest and SVM 

# importing python libraries
from imblearn.under_sampling import TomekLinks, ClusterCentroids
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from collections import Counter
import imblearn
from sklearn.preprocessing import LabelEncoder
import collections
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline



In [34]:
# import wine dataset
path = 'D:\\Notes\\Domain applications of predictive analysis\\winequalityN.csv'
wine_dataset = pd.read_csv(path)

wine_dataset.drop(wine_dataset.columns[[0]], axis = 1, inplace = True) 
wine_dataset.columns
wine = wine_dataset

In [35]:
# Setting the quality parameter as low, high and medium
quality = wine["quality"].values
category = []
for num in quality:
    if num < 5:
        category.append("Low")
    elif num > 6:
        category.append("High")
    else:
        category.append("Medium")

[(i, category.count(i)) for i in set(category)]

[('Medium', 4974), ('High', 1277), ('Low', 246)]

In [55]:
# Appending the new catergory column to the wine dataset
category = pd.DataFrame(data=category, columns=["category"])
data = pd.concat([wine, category], axis=1)

# dropping the variable quality
data.drop(columns="quality", axis=1, inplace=True)

# dropping the records with NAN values
data = data.dropna(axis = 0, how ='any') 

print("Old data frame length:", len(wine_dataset), "\nNew data frame length:",  
       len(wine), "\nNumber of rows with at least 1 NA value: ", 
       (len(wine_dataset)-len(wine))) 

X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

Old data frame length: 6497 
New data frame length: 6463 
Number of rows with at least 1 NA value:  34


In [46]:
print(data.head())


   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.0              0.27         0.36            20.7      0.045   
1            6.3              0.30         0.34             1.6      0.049   
2            8.1              0.28         0.40             6.9      0.050   
3            7.2              0.23         0.32             8.5      0.058   
4            7.2              0.23         0.32             8.5      0.058   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 45.0                 170.0   1.0010  3.00       0.45   
1                 14.0                 132.0   0.9940  3.30       0.49   
2                 30.0                  97.0   0.9951  3.26       0.44   
3                 47.0                 186.0   0.9956  3.19       0.40   
4                 47.0                 186.0   0.9956  3.19       0.40   

   alcohol category  
0      8.8   Medium  
1      9.5   Medium  
2     10.1   Medium 

In [47]:
# Splitting the data into test and train sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=2018)



In [48]:
# Applying random forest on train data to train the model
clf = RandomForestClassifier(random_state=2018, oob_score=True)
param_dist = {"n_estimators": [50, 100, 150, 200, 250],
              'min_samples_leaf': [1, 2, 4]}
rfc_gs = GridSearchCV(clf, param_grid=param_dist, scoring='accuracy', cv=5)
rfc_gs.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=True, random_state=2018,
                                 

In [49]:
# Accuracy of random forest model after training on test data 
rfc_gs.best_score_

0.8468085106382979

In [50]:
# strtifiedKFold
skf = StratifiedKFold(n_splits=4)

In [51]:
# Applying SVM on train data to train the model
pipe_svm = Pipeline([('clf', svm.SVC())])
grid_params = dict(clf__C=[0.1, 0.3, 1, 3, 10],
                   clf__gamma=[0.1, 0.3, 1, 3, 10],
                   clf__kernel=['rbf', 'sigmoid'])
gs_svm = GridSearchCV(estimator=pipe_svm,
                      param_grid=grid_params,
                      scoring='accuracy',
                      cv=skf)
gs_svm.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=4, random_state=None, shuffle=False),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('clf',
                                        SVC(C=1.0, break_ties=False,
                                            cache_size=200, class_weight=None,
                                            coef0=0.0,
                                            decision_function_shape='ovr',
                                            degree=3, gamma='scale',
                                            kernel='rbf', max_iter=-1,
                                            probability=False,
                                            random_state=None, shrinking=True,
                                            tol=0.001, verbose=False))],
                                verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'clf__C': [0.1, 0.3, 1, 3, 10],
                 

In [52]:
# Accuracy of SVM model after training on test data 
print(gs_svm.best_score_)

0.8226307588611217


In [53]:
# performance random forest on test data 
pred_rfc = rfc_gs.predict(X_test)
print(classification_report(y_test, pred_rfc))
print("The RF model accuracy on Test data is %s" %
      accuracy_score(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.82      0.59      0.69       263
           1       0.83      0.11      0.19        46
           2       0.87      0.97      0.91       984

    accuracy                           0.86      1293
   macro avg       0.84      0.55      0.60      1293
weighted avg       0.85      0.86      0.84      1293

The RF model accuracy on Test data is 0.8584686774941995


In [54]:
# performance of SVM on test data
svm_predicted = gs_svm.predict(X_test)
print(classification_report(y_test, svm_predicted))
print("---------------------------------------------------------")
print("The accuracy score of SVM (resampled) on test is %s" %
      accuracy_score(y_test, svm_predicted))

              precision    recall  f1-score   support

           0       0.98      0.32      0.49       263
           1       1.00      0.07      0.12        46
           2       0.82      1.00      0.90       984

    accuracy                           0.83      1293
   macro avg       0.93      0.46      0.50      1293
weighted avg       0.86      0.83      0.79      1293

---------------------------------------------------------
The accuracy score of SVM (resampled) on test is 0.8275328692962104
